In [1]:
import os
import numpy as np
import pandas as pd
import math
import csv

import transformers
from tqdm.notebook import trange, tqdm

from sentence_transformers import SentenceTransformer, SentencesDataset, losses
from sentence_transformers.readers import STSDataReader, TripletReader
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator, BinaryEmbeddingSimilarityEvaluator, SequentialEvaluator
from sentence_transformers.readers.InputExample import InputExample

import torch
from torch.utils.data import DataLoader, RandomSampler

from scipy.spatial.distance import cdist


In [2]:
# evaluate final mdoel on train data
"""
this_model_path = 'msmarco/models/test_model4/model4'
this_model =  SentenceTransformer('roberta-large-nli-stsb-mean-tokens') #SentenceTransformer(this_model_path)
this_train_dataloader = torch.load(os.path.join('msmarco/models/test_model4', 'train_dataloader.pth'))

evaluator1 = BinaryEmbeddingSimilarityEvaluator(this_train_dataloader)
evaluator2 = EmbeddingSimilarityEvaluator(this_train_dataloader)
this_evaluator = SequentialEvaluator([evaluator1, evaluator2])

this_model.evaluate(this_evaluator, output_path=os.path.join(this_model_path, 'train_set_performance_beforetrain'))
"""

"\nthis_model_path = 'msmarco/models/test_model4/model4'\nthis_model =  SentenceTransformer('roberta-large-nli-stsb-mean-tokens') #SentenceTransformer(this_model_path)\nthis_train_dataloader = torch.load(os.path.join('msmarco/models/test_model4', 'train_dataloader.pth'))\n\nevaluator1 = BinaryEmbeddingSimilarityEvaluator(this_train_dataloader)\nevaluator2 = EmbeddingSimilarityEvaluator(this_train_dataloader)\nthis_evaluator = SequentialEvaluator([evaluator1, evaluator2])\n\nthis_model.evaluate(this_evaluator, output_path=os.path.join(this_model_path, 'train_set_performance_beforetrain'))\n"

In [3]:
torch.cuda.empty_cache()

In [4]:
#my_model_path = 'msmarco/models/test_model4'

In [5]:
# load model
model_1 = SentenceTransformer('roberta-large-nli-stsb-mean-tokens')

In [6]:
hex(id(model_1))

'0x7f138c981198'

In [7]:
dev_dataloader = torch.load(os.path.join('/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/test_model_5', 'dev_dataloader.pth'))

In [8]:
#train_dataloader =  torch.load(os.path.join('/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/test_model_5', 'train_dataloader.pth'))

In [9]:
evaluator1 = BinaryEmbeddingSimilarityEvaluator(dev_dataloader)
evaluator2 = EmbeddingSimilarityEvaluator(dev_dataloader)
evaluator = SequentialEvaluator([evaluator1, evaluator2])

In [10]:
model_1.evaluate(evaluator, output_path=os.path.join('/run/media/root/Windows/Users/agnes/Downloads/data/msmarco/train_results/test_model_5', 
                                                     'dev_set_performance_pretrain'))

Evaluating: 100%|██████████| 250/250 [11:39<00:00,  2.80s/it]


0.6923595715624811

In [9]:
optimizer_class = transformers.AdamW
optimizer_params = {'lr': 2e-4, 'eps': 1e-6, 'correct_bias': False}
train_loss = losses.CosineSimilarityLoss(model=model_1)

In [ ]:
num_epochs = 2
warmup_steps = math.ceil(len(train_dataloader.dataset)*num_epochs / train_dataloader.batch_size*0.05) #5% of train data for warm-up

model_1.fit(train_objectives=[(train_dataloader, train_loss)],
          evaluator=evaluator,
          epochs=num_epochs,
          steps_per_epoch=100000,
          warmup_steps=warmup_steps,
          optimizer_class=optimizer_class,
          optimizer_params=optimizer_params,
          output_path=os.path.join(my_model_path, 'model3')) # works only when you have an evaluator

Iteration: 100%|██████████| 1000/1000 [18:12<00:00,  1.09s/it]

Iteration: 100%|██████████| 1000/1000 [18:09<00:00,  1.09s/it]

Iteration:  52%|█████▏    | 523/1000 [09:30<08:27,  1.06s/it]

In [ ]:
model_1.evaluate(evaluator)

In [ ]:
model_1.save(os.path.join(my_model_path, 'model'))